## **Prompting with Gemini**

In [ ]:
%%capture
!pip install google-cloud-aiplatform
!pip install -U google-generativeai

In [ ]:
import os
import json
import google.generativeai as genai
from vertexai.generative_models import GenerativeModel, Part, Image

In [ ]:
project_name = "<project_name>" # ใส่ชื่อ project ที่นี้
credentials = service_account.Credentials.from_service_account_file("/path/to/credential.json") # ใส่ path ไปยัง JSON file ที่นี่
aiplatform.init(project=project_name, credentials=credentials)

In [ ]:
def get_response(prompt, model_name="gemini-1.5-flash"):
    """
    ฟังก์ชันสำหรับรับคำตอบจากโมเดล AI
    
    Args:
    prompt (str): คำถามหรือข้อความที่ต้องการให้ AI ตอบ
    model_name (str): ชื่อของโมเดลที่ต้องการใช้ (ค่าเริ่มต้นคือ "gemini-1.5-flash")
    
    Returns:
    str: ข้อความตอบกลับจาก AI
    """
    model = GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text

In [ ]:
get_response("Why is sky blue?")

## **Multimodal Prompting with Gemini**

เราสามารถใช้ Multimodal capability ของ Gemini เพื่อใช้ทำ OCR แบบรวดเร็วเพื่อวัดผลเบื้องต้นได้

อ่านเพิ่มเติม:
- https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-gemini-all-modalities#generativeaionvertexai_gemini_all_modalities-python
- https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.vision_models.Image

In [ ]:
def get_response_with_image(prompt, image_path, model_name="gemini-pro-vision"):
    """
    ฟังก์ชันสำหรับรับคำตอบจากโมเดล AI โดยใช้รูปภาพประกอบ
    
    Args:
    prompt (str): คำถามหรือข้อความที่ต้องการให้ AI ตอบ
    image_path (str): พาธของไฟล์รูปภาพ
    model_name (str): ชื่อของโมเดลที่ต้องการใช้ (ค่าเริ่มต้นคือ "gemini-pro-vision")
    
    Returns:
    str: ข้อความตอบกลับจาก AI
    """
    model = GenerativeModel(model_name)
    image = Image.load_from_file(image_path)
    # สร้างคำตอบจากโมเดล
    response = model.generate_content([image, prompt])
    return response.text

In [ ]:
prompt = """You are provided with a scanned or photographed image of a Thai vehicle registration book (สมุดทะเบียนรถ). Your task is to extract the following information from the image.The extracted value is typically located on the right side of the key in the document.
Extract these details:

1. วันจดทะเบียน (date_of_registration)
2. เลขทะเบียน (registration_no)
3. จังหวัด (car_province)
4. ประเภท (vehicle_use)
5. รย. (type)
6. ลักษณะ (body_style)
7. ยี่ห้อรถ (manufacturer)
8. แบบ (model)
9. รุ่นปี คศ (year)
10. สี (color)
11. เลขตัวรถ (chassis_number)
12. อยู่ที่ (chassis_location)
13. ยี่ห้อเครื่องยนต์ (engine_manufacturer)
14. เลขเครื่องยนต์ (engine_number)
15. อยู่ที่ (engine_location)
16. เชื้อเพลิง (fuel_type)
17. เลขถังแก๊ส (fuel_tank_number)
18. จำนวน (cylinders)
19. ซีซี (cubic_capacity)
20. แรงม้า (horse_power)
21. จำนวนเพลาและล้อ (axles_wheels_no)
22. น้ำหนักรถ (unladen_weight)
23. น้ำหนักบรรทุก/น้ำหนักเพลา (load_capacity)
24. น้ำหนักรวม (gross_weight)
25. ที่นั่ง (seats)

Instructions:

Carefully examine the image and locate each piece of information.
If a particular field is not visible or not present in the image, use the value "N/A" for that field.
Ensure all text extracted from the image is in its original language (Thai or English) as it appears in the document.
Return the extracted information in a JSON format, using the English key names provided in parentheses.
Only return the JSON output, without any additional explanation or text.

Example of expected output format:
{
  "date_of_registration": "1 ม.ค. 2566",
  "registration_no": "กข 1234",
  "car_province": "กรุงเทพมหานคร",
  ...
  "seats": "4"
}
"""

In [ ]:
print(get_response_with_image(prompt, image_path))

## Run on all images

In [ ]:
import json
from tqdm.auto import tqdm
from glob import glob
from pathlib import Path
import pandas as pd

extracted_values = []
error_paths = []
paths = glob("/Users/titipata/Desktop/Misc_docs/OCR_Skooldio/Srisawad_Dataset/*.jpg")
for path in tqdm(paths):
    extracted_information = get_response_with_image(prompt, path)
    print(f"path {path} is processed")
    try:
        extracted_information = extracted_information.replace("```json", "").replace("```", "")
        extracted_information = json.loads(extracted_information)
        extracted_values.append(extracted_information)
    except:
        error_paths.append(path)
        print(f"Error processing: {path}")
extracted_values_df = pd.DataFrame(extracted_values)

In [ ]:
extracted_values_df["image_path"] = [Path(p).stem for p in paths]
extracted_values_df.to_excel("predicted_results_gemini.xlsx", index=False)
# Preview the DataFrame
display(extracted_values_df)

## Evaluation

In [ ]:
import numpy as np
from torchmetrics.text import CharErrorRate

def calculate_cer(preds: list, targets: list):
    cer = CharErrorRate() # Initialize the CharErrorRate metric
    cer_val = cer(preds, targets) # Calculate CER
    return cer_val.tolist()

In [ ]:
annotated_df = pd.read_excel('annotated_results.xlsx', dtype=str).fillna("")
predicted_df = pd.read_excel('predicted_results_gemini.xlsx', dtype=str).fillna("")

columns_of_interest = [
    'date_of_registration', 'registration_no', 'car_province', 'vehicle_use', 'type', 'body_style',
    'manufacturer', 'model', 'year', 'color', 'chassis_number', 'chassis_location', 'engine_manufacturer',
    'engine_number', 'engine_location', 'fuel_type', 'fuel_tank_number', 'cylinders', 'cubic_capacity',
    'horse_power', 'axles_wheels_no', 'unladen_weight', 'load_capacity', 'gross_weight', 'seats'
]
merged_df = pd.merge(annotated_df, predicted_df, on='image_path', suffixes=('_annotation', '_prediction'))

In [ ]:
eval_list = []
for col in columns_of_interest:
    if f"{col}_annotation" in merged_df.columns and f"{col}_prediction" in merged_df.columns:
        avg_cer = np.mean(calculate_cer(merged_df[f"{col}_prediction"], merged_df[f"{col}_annotation"]))
        avg_accuracy = (merged_df[f"{col}_prediction"] == merged_df[f"{col}_annotation"]).mean() * 100
        eval_list.append({
            "column_name": col,
            "cer": avg_cer,
            "accuracy": avg_accuracy
        })
eval_df = pd.DataFrame(eval_list)

In [ ]:
display(eval_df)